In [1]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model

Using TensorFlow backend.


In [2]:
def generator(dir,
              gen=image.ImageDataGenerator(rescale=1./255),
              shuffle=True,
              batch_size=1,
              target_size=(24,24),
              class_mode='categorical' ):

    return gen.flow_from_directory(dir,
                                   batch_size=batch_size,
                                   shuffle=shuffle,
                                   color_mode='grayscale',
                                   class_mode=class_mode,
                                   target_size=target_size)

BS= 32
TS=(24,24)
train_batch= generator('a2a_data/train',
                       shuffle=True, 
                       batch_size=BS,
                       target_size=TS)
valid_batch= generator('a2a_data/valid',
                       shuffle=True, 
                       batch_size=BS,
                       target_size=TS)
SPE= len(train_batch.classes)//BS
VS = len(valid_batch.classes)//BS
print(SPE,VS)

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'a2a_data/train'

In [ ]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)), 
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
   
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', precision, recall, f1score])

model.fit_generator(train_batch, 
                    validation_data=valid_batch,
                    epochs=15,
                    steps_per_epoch=SPE,
                    validation_steps=VS)

model.save('a2a_model.h5', overwrite=True)

In [ ]:
_loss, _acc, _precision, _recall, _f1score = model.evaluate_generator(valid_batch, steps=500)
print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _precision, _recall, _f1score))